# [M_03] PROTOKÓŁ: AUTOMATYCZNA FABRYKA WIDEO (FFMPEG)

**PROJEKT:** OMNI-OPERATOR-V1  
**SILNIK:** GEMINI 2.5 FLASH  
**STATUS:** EKSTRACJA_MEDIÓW  

Ten moduł odpowiada za fizyczną obróbkę materiału wideo. Wykorzystujemy bibliotekę `MoviePy` (napędzaną przez `FFmpeg`), aby na podstawie ustrukturyzowanego raportu z Gemini 2.5 Flash automatycznie wyciąć i wyrenderować fragmenty pod Shortsy.

**Cele operacyjne:**
1. Konwersja znaczników czasu `MM:SS` na sekundy (float).
2. Automatyczna ekstrakcja klipów z pliku źródłowego `test_video.mp4`.
3. Zastosowanie parametrów renderowania zoptymalizowanych pod urządzenia mobilne (H.264).

In [1]:
import os
import sys
from moviepy.video.io.VideoFileClip import VideoFileClip

# 1. KOREKTA ŚCIEŻKI ROBOCZEJ
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Dodanie src do path, aby widzieć ewentualne moduły pomocnicze
sys.path.append(os.path.join(os.getcwd(), "src"))

print(f"LOG: System montażowy zainicjowany. Katalog ROOT: {os.getcwd()}")

LOG: System montażowy zainicjowany. Katalog ROOT: c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1


## 1. Narzędzia Pomocnicze: Konwersja Czasu

Raport z Gemini (M_01) dostarcza czas w formacie tekstowym `MM:SS`. Silnik FFmpeg wymaga precyzyjnej liczby sekund. Implementujemy bezpieczny konwerter.

In [2]:
def timestamp_to_seconds(ts: str) -> float:
    """Konwertuje format MM:SS na sekundy (np. '01:15' -> 75.0)."""
    try:
        parts = ts.split(':')
        if len(parts) == 2:
            minutes, seconds = map(int, parts)
            return float(minutes * 60 + seconds)
        return 0.0
    except (ValueError, AttributeError):
        print(f"ERR: Błędny format znacznika czasu: {ts}")
        return 0.0

# TEST OPERACYJNY
test_ts = "00:45"
print(f"TEST: {test_ts} -> {timestamp_to_seconds(test_ts)}s")

TEST: 00:45 -> 45.0s


## 2. Implementacja Silnika Produkcyjnego

Tworzymy funkcję `produce_shorts`, która bierze surowy film i listę klipów, a następnie wykonuje renderowanie. Każdy klip zostanie zapisany w dedykowanym folderze `output/`.

In [5]:
def produce_shorts(source_file: str, clips_to_cut: list):
    """
    Fizycznie wycina fragmenty wideo na podstawie listy słowników.
    Oczekiwany format: {'start': 'MM:SS', 'end': 'MM:SS'}
    Zgodne z MoviePy v2.x
    """
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    
    if not os.path.exists(source_file):
        raise FileNotFoundError(f"Nie znaleziono pliku źródłowego: {source_file}")

    print(f"LOG: Otwieram materiał źródłowy: {source_file}")
    
    # W MoviePy 2.x używamy menedżera kontekstu (with)
    with VideoFileClip(source_file) as video:
        for i, clip_info in enumerate(clips_to_cut, 1):
            start_s = timestamp_to_seconds(clip_info['start'])
            end_s = timestamp_to_seconds(clip_info['end'])
            
            output_path = os.path.join(output_dir, f"short_{i}.mp4")
            
            print(f"LOG: Renderowanie klipu {i} [{clip_info['start']} - {clip_info['end']}]...")
            
            # NOWA METODA W v2.x: subclipped() zamiast subclip()
            new_clip = video.subclipped(start_s, end_s)
            
            # Zapis pliku
            new_clip.write_videofile(
                output_path, 
                codec="libx264", 
                audio_codec="aac",
                logger=None # Wyłączamy pasek postępu dla czystości logów
            )
            print(f"✅ GOTOWE: {output_path}")

    print(f"\nLOG: Produkcja zakończona pomyślnie.")

## 3. Uruchomienie Fabryki (Test End-to-End)

Używamy danych uzyskanych z Gemini 2.5 Flash w Module 01, aby wygenerować pierwsze pliki.

In [6]:
# Symulacja danych z raportu analitycznego (M_01)
# Jeśli masz już analysis_result.clips, możesz użyć tych danych bezpośrednio
mock_production_data = [
    {"start": "00:00", "end": "00:03", "hook": "Efekt wizualny AI"},
    {"start": "00:03", "end": "00:06", "hook": "Prezentacja frameworka"}
]

INPUT_VIDEO = "test_video.mp4"

try:
    produce_shorts(INPUT_VIDEO, mock_production_data)
except Exception as e:
    print(f"❌ BŁĄD PRODUKCJI: {str(e)}")

LOG: Otwieram materiał źródłowy: test_video.mp4
LOG: Renderowanie klipu 1 [00:00 - 00:03]...
✅ GOTOWE: output\short_1.mp4
LOG: Renderowanie klipu 2 [00:03 - 00:06]...
✅ GOTOWE: output\short_2.mp4

LOG: Produkcja zakończona pomyślnie.


## STATUS: MODUŁ 03 ZAKOŃCZONY

Mamy teraz działającą fabrykę wideo. System potrafi autonomicznie zarządzać plikami na dysku.

**Osiągnięcia:**
1. Integracja FFmpeg z logicznym workflow agenta.
2. Automatyczne generowanie plików MP4 zoptymalizowanych pod social media.
3. Pełna suwerenność nad procesem montażu.

**Zadanie:**
Otwórz folder `output/` w swoim projekcie i sprawdź, czy pliki `short_1.mp4` i `short_2.mp4` odtwarzają się poprawnie.